In [1]:
import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

In [18]:
no_of_future_frames = 5
no_of_past_frames = 5

In [2]:
def iou_batch(bb_test, bb_gt):
  """
  From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
  """
  bb_gt = np.expand_dims(bb_gt, 0)
  bb_test = np.expand_dims(bb_test, 1)
  
  xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
  yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
  xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
  yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
  return(o)  


def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))

def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))

In [20]:
class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = collections.deque(maxlen=no_of_past_frames)
    self.hits = 0
    self.hit_streak = 0
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
#     self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)

In [19]:
import collections
l = collections.deque(maxlen=no_of_past_frames)
print(len(l))

0


In [11]:
# predict and update simultaniously
f = KalmanBoxTracker([0,1,2,3])
# fc = KalmanBoxTracker(f.history[-1][0])
true=[]
predicted=[]

for i in range(5):
    z = [i+1, i+2, i+3, i+4]
    true.append(z)
    p = f.predict()[0]
    f.update(z)
    predicted.append(p[:2])
    print('len history ', len(f.history))
    print('predicted :\n', p)
    print('Measured : ', z)
    plt.plot(z[0],z[1], c='blue', marker='o', markersize=6)
    plt.plot(p[0],p[1], c='red', marker='*', markersize=3)
# plt.show()

len history  1
predicted :
 [0. 1. 2. 3.]
Measured :  [1, 2, 3, 4]
len history  2
predicted :
 [1.99870156 2.99870156 3.99870156 4.99870156]
Measured :  [2, 3, 4, 5]
len history  3
predicted :
 [2.99971793 3.99971793 4.99971793 5.99971793]
Measured :  [3, 4, 5, 6]
len history  4
predicted :
 [3.99987343 4.99987343 5.99987343 6.99987343]
Measured :  [4, 5, 6, 7]
len history  5
predicted :
 [4.99992445 5.99992445 6.99992445 7.99992445]
Measured :  [5, 6, 7, 8]


In [68]:
# predict 5 and update 5
f = KalmanBoxTracker([0,1,2,3])
# fc = KalmanBoxTracker(f.history[-1][0])
true=[]
predicted_All=[]
predicted = collections.deque(maxlen=no_of_past_frames)
future_bb = []

for i in range(1,16):
    
    z = [i, i+1, i+2, i+3]
    true.append(z)
    p = f.predict()[0]
    f.update(z)
    predicted.append(p[:2])
    predicted_All.append(p[:2])
    print('\nlen history ', len(f.history))
    print('hits counter ', f.hits, ' hit_streak ',f.hit_streak,' age ',f.age)
    print('predicted :\n', p)
    print('Measured : ', z)
    plt.plot(z[0],z[1], c='blue', marker='o', markersize=6)
    plt.plot(p[0],p[1], c='red', marker='*', markersize=3)
    
    if  i%no_of_past_frames == 0 and i>1:
        print('\n--------- %s predicted future_frames: ---------\n'%i)
        fc = KalmanBoxTracker(f.history[0][0])
        
        for j in range(1,no_of_past_frames):
            fc.predict()
            p = fc.update(f.history[j][0])
            
        for k in range(no_of_future_frames):
            p = fc.predict()[0]
            plt.plot(p[0],p[1], c='black', marker='*', markersize=1)
            print('predicted future_frames:\n', p)
            
        print('\n--------- %s end of predicted future_frames: ---------\n'%i)
            
    
        
plt.show()


len history  1
hits counter  1  hit_streak  1  age  1
predicted :
 [0. 1. 2. 3.]
Measured :  [1, 2, 3, 4]

len history  2
hits counter  2  hit_streak  2  age  2
predicted :
 [1.99870156 2.99870156 3.99870156 4.99870156]
Measured :  [2, 3, 4, 5]

len history  3
hits counter  3  hit_streak  3  age  3
predicted :
 [2.99971793 3.99971793 4.99971793 5.99971793]
Measured :  [3, 4, 5, 6]

len history  4
hits counter  4  hit_streak  4  age  4
predicted :
 [3.99987343 4.99987343 5.99987343 6.99987343]
Measured :  [4, 5, 6, 7]

len history  5
hits counter  5  hit_streak  5  age  5
predicted :
 [4.99992445 5.99992445 6.99992445 7.99992445]
Measured :  [5, 6, 7, 8]

--------- 5 predicted future_frames: ---------

predicted future_frames:
 [6.09847132 7.09847132 8.09847132 9.09847132]
predicted future_frames:
 [ 7.16351208  8.16351208  9.16351208 10.16351208]
predicted future_frames:
 [ 8.22855285  9.22855285 10.22855285 11.22855285]
predicted future_frames:
 [ 9.29359361 10.29359361 11.29359361 1

In [ ]:
# predict future positions if past position limit {no_of_past_frames} is reached
f = KalmanBoxTracker([0,1,2,3])
# fc = KalmanBoxTracker(f.history[-1][0])
true=[]
predicted_All=[]
predicted = collections.deque(maxlen=no_of_past_frames)
future_bb = []

for i in range(1,16):
    
    z = [i, i+1, i+2, i+3]
    true.append(z)
    p = f.predict()[0]
    f.update(z)
    predicted.append(p[:2])
    predicted_All.append(p[:2])
    print('\nlen history ', len(f.history))
    print('predicted :\n', p)
    print('Measured : ', z)
    plt.plot(z[0],z[1], c='blue', marker='o', markersize=6)
    plt.plot(p[0],p[1], c='red', marker='*', markersize=3)
    
    if  len == no_of_past_frames:
        print('\n--------- %s predicted future_frames: ---------\n'%i)
        fc = KalmanBoxTracker(f.history[0][0])
        
        for j in range(1,no_of_past_frames):
            fc.predict()
            p = fc.update(f.history[j][0])
            
        for k in range(no_of_future_frames):
            p = fc.predict()[0]
            plt.plot(p[0],p[1], c='black', marker='*', markersize=1)
            print('predicted future_frames:\n', p)
            
        print('\n--------- %s end of predicted future_frames: ---------\n'%i)
            
    
        
plt.show()